In [106]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timezone 

In [79]:
# Check your list of packages you've installed in conda.
!conda list

# packages in environment at C:\anaconda3\envs\pytorch_learning:
#
# Name                    Version                   Build  Channel
anyio                     3.5.0           py310haa95532_0    anaconda
argon2-cffi               21.3.0             pyhd3eb1b0_0    anaconda
argon2-cffi-bindings      21.2.0          py310h2bbff1b_0    anaconda
asttokens                 2.0.5              pyhd3eb1b0_0    anaconda
attrs                     22.1.0          py310haa95532_0    anaconda
babel                     2.11.0          py310haa95532_0    anaconda
backcall                  0.2.0              pyhd3eb1b0_0    anaconda
beautifulsoup4            4.11.1          py310haa95532_0    anaconda
blas                      1.0                         mkl  
bleach                    4.1.0              pyhd3eb1b0_0    anaconda
bottleneck                1.3.5           py310h9128911_0    anaconda
brotli                    1.0.9                h2bbff1b_7    anaconda
brotli-bin                1.0.9     

In [140]:
# load the dataset from Kaggle
df = pd.read_csv('dataset_71k.csv')

In [116]:
# visualize dataset properties
print("Dataset properties before processing:")
print("Entries:", len(df))
print("Columns:", len(df.columns))
print("Shape:", df.shape)
print("Include columns:", list(df))

Dataset properties before processing:
Entries: 71250
Columns: 18
Shape: (71250, 18)
Include columns: ['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_timestamp', 'block_number', 'block_hash', 'from_scam', 'to_scam', 'from_category', 'to_category']


In [254]:
data_df = pd.read_json('txn_data.json')

C:\anaconda3\envs\pytorch_learning\lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [ ]:
data_df.to_csv("blockscout_txn_output.csv", index=False)

In [255]:
# setting up the dataset
# run here if txn and addr data are present
txn_df = pd.read_csv('blockscout_txn_output.csv')
merged_df = df.merge(txn_df, on='hash')
merged_df.to_csv("merged_data.csv", index=False)

In [256]:
merged_df['from_address']

0        0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0
1        0x304cc179719bc5b05418d6f7f6783abe45d83090
2        0x304cc179719bc5b05418d6f7f6783abe45d83090
3        0x304cc179719bc5b05418d6f7f6783abe45d83090
4        0x304cc179719bc5b05418d6f7f6783abe45d83090
                            ...                    
71267    0x7c9001c50ea57c1b2ec1e3e63cf04c297534bfc1
71268    0xf49874712828e3d73a12c15f9f77c5529eb0632f
71269    0x7c9001c50ea57c1b2ec1e3e63cf04c297534bfc1
71270    0xf49874712828e3d73a12c15f9f77c5529eb0632f
71271    0x7c9001c50ea57c1b2ec1e3e63cf04c297534bfc1
Name: from_address, Length: 71272, dtype: object

In [257]:
data_df = pd.read_json('addr_data.json')
data_df.to_csv("blockscout_addr_output.csv", index=False)

In [258]:
addr_df = pd.read_csv('blockscout_addr_output.csv')
addr_df['hash'] = addr_df['hash'].str.lower()

C:\Users\Matthew Ko\AppData\Local\Temp\ipykernel_6728\3368176022.py:1: DtypeWarning: Columns (2,3,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  addr_df = pd.read_csv('blockscout_addr_output.csv')


In [259]:
merged_df_1 = merged_df.merge(addr_df, how='left', left_on='from_address', right_on='hash')
merged_df_2 = merged_df_1.merge(addr_df, how='left', left_on='to_address', right_on='hash')

In [261]:
merged_df_2.to_csv("merged_dataset.csv", index=False)

In [262]:
df = pd.read_csv("merged_dataset.csv")
df.head()

C:\Users\Matthew Ko\AppData\Local\Temp\ipykernel_6728\1547917139.py:1: DtypeWarning: Columns (84,85,86,87,88,89,90,91,92,93,94,95,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged_dataset.csv")


,hash_x,nonce_x,transaction_index,from_address,to_address,value_x,gas,gas_price_x,input,receipt_cumulative_gas_used,...,implementation_address_y,implementation_name_y,is_contract_y,is_verified_y,name_y,private_tags_y,public_tags_y,token_y,watchlist_address_id_y,watchlist_names_y
0,0x253ec84729f5c11437f5346830e6bdc9857171b16097...,370394,4,0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0,0xd707ac0098599006f0857e8da4c950795645ba01,1.800000e+18,30000,2.205000e+10,0x,129358,...,NaN,NaN,False,False,NaN,[],[],NaN,NaN,[]
1,0xfa5f1d7715562adb1e408fdbbbbfc033ae4fadca1e07...,229082,85,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xb8f6e76d34d4877732dfefa6b6a4e2a834a9092b,7.500000e+15,90000,2.060947e+10,0x,5415387,...,NaN,NaN,False,False,NaN,[],[],NaN,NaN,[]
2,0x37928f1d12ee769d0876695aaa815c13d705f8b1e376...,229095,31,0x304cc179719bc5b05418d6f7f6783abe45d83090,0x5ec492652d7b92421680f21169ff6a8dbbc622ab,7.500000e+15,90000,2.060947e+10,0x,1369625,...,NaN,NaN,False,False,NaN,[],[],NaN,NaN,[]
3,0xaad886f331a2fb40495cdf5faa10882852fcd324f064...,229126,78,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xdf01b231b53d8b4cbb1b092a8dc86730cebf4aa9,7.500000e+15,90000,2.060947e+10,0x,2464655,...,NaN,NaN,False,False,NaN,[],[],NaN,NaN,[]
4,0x3dbe36fd1a1b01b5e1b343ff78ec1ec189feee5acf46...,229131,83,0x304cc179719bc5b05418d6f7f6783abe45d83090,0xaa4b9693c54029d180134161cc99e98b76cea249,7.500000e+15,90000,2.060947e+10,0x,2569655,...,NaN,NaN,False,False,NaN,[],[],NaN,NaN,[]


In [75]:
# change existing column

def calculate_elapsed_time(row):
    elapsed_time = pd.Timestamp.utcnow() - row['timestamp_obj']
    return elapsed_time

merged_df['is_scam'] = merged_df.apply(lambda row: 1 if row['from_scam'] == 1 or row['to_scam'] == 1 else 0, axis=1)
merged_df['timestamp_obj'] = pd.to_datetime(merged_df['block_timestamp'])
merged_df['elapsed_time'] = merged_df.apply(lambda row: calculate_elapsed_time(row), axis=1)